---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [270]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import recall_score, precision_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from adspy_shared_utilities import plot_class_regions_for_classifier_subplot
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing


In [271]:
# test_df = pd.read_csv('readonly/test.csv')
# addresses_df = pd.read_csv('readonly/addresses.csv')
# latlons_df = pd.read_csv('readonly/latlons.csv')



In [272]:
# train_df = pd.read_csv('readonly/train.csv', engine='python')
#

In [273]:
# train_df.to_csv (r'train_2.csv', index = False, header=True)


In [274]:
# test_df.to_csv (r'test_2.csv', index = False, header=True)
# addresses_df.to_csv (r'addresses_2.csv', index = False, header=True)
# latlons_df.to_csv (r'latlons_2.csv', index = False, header=True)


In [275]:

# train_df = pd.read_csv('train_2.csv')
# test_df = pd.read_csv('test_2.csv')
# latlons_df = pd.read_csv('latlons_2.csv')
# address_df = pd.read_csv('addresses_2.csv')


train_df = pd.read_csv('train.csv', engine='python')
test_df = pd.read_csv('test.csv', engine='python')
latlons_df = pd.read_csv('latlons.csv', engine='python')
address_df = pd.read_csv('addresses.csv', engine='python')

# train_df

In [276]:
X_df = train_df.copy()
col = ['payment_amount','payment_date','payment_status','balance_due','collection_status','compliance','compliance_detail']
y_train = X_df.loc[:,['compliance']] 
X_train = X_df.drop( col , axis='columns')

In [277]:
# test_df

In [278]:
# X_train.head()
# y_train

In [279]:
X_test = test_df.copy()

In [280]:
y = X_test.copy()
y[''] = 0.000000
y_test = y.iloc[:,-1]
# y_test

In [281]:
X_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,Failure of owner to obtain certificate of comp...,Not responsible by City Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,Failure of owner to obtain certificate of comp...,Not responsible by Dismissal,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [256]:
le = LabelEncoder()
# le.fit(X_train['grafitti_status','disposition'].astype(str))
# X_train['grafitti_status'] = le.transform(X_train['grafitti_status'].astype(str))
# X_test['grafitti_status'] = le.transform(X_test['grafitti_status'].astype(str))

In [257]:
# le.fit(X_train['disposition'].astype(str))
# X_train['disposition'] = le.transform(X_train['disposition'].astype(str))
# X_test['disposition'] = le.transform(X_test['disposition'].astype(str))

In [258]:
# df[cat] = le.fit_transform(df[cat].astype(str))
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna('nothing', inplace=True)
y_train.fillna(2, inplace=True)

X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.fillna('nothing', inplace=True)

for column_name in X_train.columns:
    if X_train[column_name].dtype == object:
        X_train[column_name] = le.fit_transform(X_train[column_name].astype(str))
    else:
        pass

for column_name in X_test.columns:
    if X_test[column_name].dtype == object:
        X_test[column_name] = le.fit_transform(X_test[column_name].astype(str))
    else:
        pass

In [259]:
# X_train.replace([np.inf, -np.inf], 999, inplace=True)
# X_train.fillna(0, inplace=True)

In [260]:
X_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,22056,0,137,52230,2900.0,1640,0,9011,27078,682,...,77,6,20,20.0,10.0,25.0,0.0,0.0,305.0,1
1,27586,0,166,77135,4311.0,306,0,8896,18442,1204,...,54,7,39,20.0,10.0,75.0,0.0,0.0,855.0,1
2,22062,0,137,94185,1449.0,1021,0,6994,21873,1104,...,77,2,20,0.0,0.0,0.0,0.0,0.0,0.0,1
3,22084,0,137,74625,1441.0,1021,0,11980,28889,1104,...,77,0,20,0.0,0.0,0.0,0.0,0.0,0.0,1
4,22093,0,137,77944,2449.0,343,0,13939,5401,1104,...,77,2,20,0.0,0.0,0.0,0.0,0.0,0.0,1


In [261]:

x = X_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X_train = pd.DataFrame(x_scaled)

In [262]:
# fit = GaussianNB().fit(X_train, y_train)

In [ ]:
# default RandomForest
# RandomForestClassifier(bootstrap=True, class_weight=None, criterion=’gini’,
# max_depth=None, max_features=’auto’, max_leaf_nodes=None,
# min_impurity_split=1e-07, min_samples_leaf=1,
# min_samples_split=2, min_weight_fraction_leaf=0.0,
# n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
# verbose=0, warm_start=False)

In [263]:
# clf = RandomForestClassifier(n_estimators=10, max_depth=6, n_jobs=1, verbose=2)


# fit_RF_auc_o69_score_76 = RandomForestClassifier(n_estimators = 10,max_depth=6, n_jobs=1, verbose=2,
#                             random_state=0).fit(X_train, y_train)

# tune max_features
# fit_RF_auc_o69_score_76 = RandomForestClassifier(n_estimators = 10,max_depth=6, n_jobs=1, verbose=2, max_features= 15,
#                             random_state=0).fit(X_train, y_train)

# tune n_estimators
fit_RF_auc_o69_score_76 = RandomForestClassifier(n_estimators = 15, max_depth=6, n_jobs=1, verbose=2, max_features= 15,
                            random_state=0).fit(X_train, y_train)

# tune max_depth
# fit_RF_auc_o69_score_76  = RandomForestClassifier(n_estimators = 15, max_depth=8, n_jobs=1, verbose=2, max_features= 15,
#                             random_state=0).fit(X_train, y_train)

# tune max_features
fit_RF_auc_o74_score_96 = RandomForestClassifier(n_estimators = 15, max_depth=8, n_jobs=1, verbose=2, max_features= 10,
                            random_state=0).fit(X_train, y_train)


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


building tree 1 of 10


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    3.3s finished


In [264]:
x_2 = X_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled_2 = min_max_scaler.fit_transform(x_2)
X_test = pd.DataFrame(x_scaled_2)

y_pred = fit.predict_proba(X_test)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [265]:
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.head()
y_array = y_pred_df[1].values
y_array

array([ 0.1245413 ,  0.02088789,  0.1245413 , ...,  0.11997509,
        0.11997509,  0.16653754])

In [266]:
test_df.head()
x_array = test_df.ticket_id.values
x_array

array([284932, 285362, 285361, ..., 376499, 376500, 369851])

In [267]:
s = pd.Series(y_array,x_array)
ans = pd.DataFrame(s)
# ans = ans.reset_index()
# ans.rename(index={0:'ticket_id'}, inplace=True)
ans.index.names = ['ticket_id']
ans

,0
ticket_id,
284932,0.124541
285362,0.020888
285361,0.124541
285338,0.121225
285346,0.131123
285345,0.121225
285347,0.131123
285342,0.154823
285530,0.020888


In [268]:


def blight_model():
    
    
    
    # Your code here
    
    return ans# Your answer here

In [246]:
# print(blight_model())
blight_model()

,0
ticket_id,
284932,0.124541
285362,0.020888
285361,0.124541
285338,0.121225
285346,0.131123
285345,0.121225
285347,0.131123
285342,0.154823
285530,0.020888
